In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from typing import Dict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embeddings = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        return self.embeddings(x) * math.sqrt(self.d_model)


class PositionEncoding(nn.Module):
    def __init__(self, d_model: int, seq_len: int, dropout: float | None):
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = None
        if dropout is not None:
            self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(seq_len, d_model)  # positional encoding blueprint matrix
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)  # numerator in the formula, shape=(seq_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()*(-math.log(10000.0)/d_model)) # for numerical stability using log space
        pe[:, 0::2] = torch.sin(position*div_term)  # at 0, 2, ..., 2k, ...
        pe[:, 1::2] = torch.cos(position*div_term)  # at 1, 3, ..., 2k+1, ...
        self.pe = pe.unsqueeze(0).to(device)  # shape=(1, seq_len, d_model)

        #self.register_buffer('pe', pe.unsqueeze(0))  # save the positional encoding in the module along with the saved model (NOT as a learned param)

    def forward(self, x):
        if self.dropout is not None:
            return self.dropout(x+(self.pe[:, :x.shape[1], :]).requires_grad_(False))  # setting requries_grad_ False ensures pe is not learned
        return x+(self.pe[:, :x.shape[1], :]).requires_grad_(False)

# attention is all you need!!
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, h: int, dropout: float | None):
        super().__init__()
        self.d_model = d_model
        self.h = h
        assert d_model%h == 0, "d_model has to be divisible by h"

        self.d_k = d_model//h
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_o = nn.Linear(d_model, d_model)
        self.dropout = None
        if dropout is not None:
            self.dropout = nn.Dropout(dropout)

        self.attention = ScaledDotProductAttention(self.d_k, self.dropout)

    def forward(self, Q, K, V, mask=None):
        # shape transition: Q,K,V=(batch_size, seq_len, d_model) -W-> (batch_size, seq_len, d_model) -> (batch_size, seq_len, h, d_k) -T-> (batch_size, h, seq_len, d_k)
        q = self.w_q(Q).view(Q.shape[0], Q.shape[1], self.h, self.d_k).transpose(1, 2)
        k = self.w_k(K).view(K.shape[0], K.shape[1], self.h, self.d_k).transpose(1, 2)
        v = self.w_v(V).view(V.shape[0], V.shape[1], self.h, self.d_k).transpose(1, 2)

        x_out, attn = self.attention(q,k,v,mask)
        # x_out=(batch_size, h, seq_len, d_k) -> (batch_size, seq_len, h, d_k) -> (batch_size, seq_len, d_model)
        x_out = x_out.transpose(1, 2).contiguous().view(x_out.shape[0], -1, self.h*self.d_k)  # contiguous to ensure the tensor is stored as contiguous blocks
        return self.w_o(x_out)  # -> (batch_size, seq_len, d_model)


class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k, dropout: nn.Dropout | None):
        super().__init__()
        self.d_k = d_k
        self.dropout = dropout
    def forward(self, Q, K, V, mask=None):
        scores = torch.matmul(Q, K.transpose(-2, -1))/math.sqrt(self.d_k)  # (batch_size, h, seq_len, d_k)
        if mask is not None:
            scores = scores.masked_fill(mask==0, -1e-9)
        if self.dropout is not None:
            scores = self.dropout(scores)
        attn = F.softmax(scores, dim=-1)  # -> (batch_size, h, seq_len, seq_len)
        return torch.matmul(attn, V), attn  # -> (batch_size, h, seq_len, d_k)


class LayerNormalization(nn.Module):
    def __init__(self, epsilon=1e-6):
        super().__init__()
        self.epsilon = epsilon
        self.gamma = nn.Parameter(torch.ones(1))  # multiplier
        self.beta = nn.Parameter(torch.zeros(1))  # bias

    def forward(self, x):
        mean = x.float().mean(dim=-1, keepdim=True)
        std = x.float().std(dim=-1, keepdim=True)
        return self.gamma*(x-mean)/(std+self.epsilon)+self.beta


class FeedForward(nn.Module):
    # formula: max(0, xW1+b1)W2+b2, where W1,2 are linear layers, b1,b2 are biases, and max(0, z) is done by a relu layer
    def __init__(self, d_model: int, d_ff: int, dropout: float | None):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff, bias=True)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(d_ff, d_model, bias=True)
        self.dropout = None
        if dropout is not None:
            self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        if self.dropout is not None:
            return self.linear2(self.dropout(self.relu(self.linear1(x))))
        return self.linear2(self.relu(self.linear1(x)))


# skip connection norm->norm aside from norm->feedforward
class ResidualConnection(nn.Module):
    def __init__(self, dropout: float | None):
        super().__init__()
        self.dropout = None
        if dropout is not None:
            self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization()

    def forward(self, x, sublayer):  # sublayer is prev layer
        if self.dropout is not None:
            return self.dropout(sublayer(self.norm(x)))
        return sublayer(self.norm(x))


# for the customizability (of dropout), pass each block to stack up instead of parameters to create blocks from scratch
class EncoderBlock(nn.Module):
    def __init__(self, self_attention: MultiHeadAttention, feed_forward: FeedForward, dropout: float | None):
        super().__init__()
        self.self_attention = self_attention
        self.feed_forward = feed_forward
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

    def forward(self, x, source_mask):
        x_out = self.residual_connections[0](x, lambda x: self.self_attention(x, x, x, source_mask))  # forward in MultiHeadAttention
        x_out = self.residual_connections[1](x_out, self.feed_forward)
        return x_out


class Encoder(nn.Module):
    def __init__(self, layers: nn.ModuleList):
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, x, mask):
        x_out = x
        for layer in self.layers:
            x_out = layer(x_out, mask)
        return self.norm(x_out)


class DecoderBlock(nn.Module):
    def __init__(self, self_attention: MultiHeadAttention, cross_atention: MultiHeadAttention, feed_forward: FeedForward, dropout: float | None):
        super().__init__()
        self.self_attention = self_attention
        self.cross_attention = cross_atention
        self.feed_forward = feed_forward
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])

    def forward(self, x, encoder_output, source_mask, target_mask):
        x_out = self.residual_connections[0](x, lambda x: self.self_attention(x, x, x, target_mask))
        x_out = self.residual_connections[1](x_out, lambda x_out: self.cross_attention(x_out, encoder_output, encoder_output, source_mask))
        x_out = self.residual_connections[2](x_out, self.feed_forward)
        return x_out


class Decoder(nn.Module):
    def __init__(self, layers: nn.ModuleList):
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, x, encoder_output, source_mask, target_mask):
        x_out = x
        for layer in self.layers:
            x_out = layer(x_out, encoder_output, source_mask, target_mask)
        return self.norm(x_out)


class ProjectionLayer(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        return torch.log_softmax(self.proj(x), dim=-1)  # (batch_size, seq_len, d_model) -> (batch_size, seq_len, vocab_size)


# only encoder - useful for classification task
class TransformerEncoder(nn.Module):
    def __init__(self, encoder: Encoder, embed: InputEmbeddings, pe: PositionEncoding, proj):
        super().__init__()
        self.encoder = encoder
        self.embed = embed
        self.pe = pe
        self.proj = proj

    def forward(self, x, mask):
        x_out = self.embed(x)
        x_out = self.pe(x_out)
        return self.proj(self.encoder(x_out, mask))


# full transformer
class Transformer(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder, source_embed: InputEmbeddings, target_embed: InputEmbeddings, source_pe: PositionEncoding, target_pe: PositionEncoding, proj: ProjectionLayer):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.source_embed = source_embed
        self.target_embed = target_embed
        self.source_pe = source_pe
        self.target_pe = target_pe
        self.proj = proj

    def encode(self, x, source_mask):
        x_out = self.source_embed(x)
        x_out = self.source_pe(x_out)
        return self.encoder(x_out, source_mask)

    def decode(self, x, encoder_output, source_mask, target_mask):
        x_out = self.target_embed(x)
        x_out = self.target_pe(x_out)
        return self.decoder(x_out, encoder_output, source_mask, target_mask)

    def forward(self, source_input, target_input, source_mask, target_mask):
        encoder_output = self.encode(source_input, source_mask)
        decoder_output = self.decode(target_input, encoder_output, source_mask, target_mask)
        return self.proj(decoder_output)


# building functions
# for encoder-only transformer
def build_encoder_transformer(vocab_size: int, seq_len: int, dropouts: Dict[str, float | None] | None, d_model: int=512, d_ff: int=2048,  N: int=6, h: int=8):
    key_list = ['encoder_pe', 'encoder_self_attetion', 'encoder_feed_forward', 'encoder_block']
    if dropouts is None:
        dropouts = dict()
    for key in key_list:
        if key not in dropouts.keys():
            dropouts[key] = None

    embed = InputEmbeddings(d_model, vocab_size)
    pos = PositionEncoding(d_model, seq_len, dropouts['encoder_pe'])
    layers = []
    for _ in range(N):
        self_attention = MultiHeadAttention(d_model, h, dropouts['encoder_self_attetion'])
        feed_forward = FeedForward(d_model, d_ff, dropouts['encoder_feed_forward'])
        layers.append(EncoderBlock(self_attention, feed_forward, dropouts['encoder_block']))
    encoder = Encoder(nn.ModuleList(layers))
    proj = nn.Linear(d_model, vocab_size)

    transformer_encoder = TransformerEncoder(encoder, embed, pos, proj)

    # Xavier uniform distribution for parameter initialization
    for param in transformer_encoder.parameters():
        if param.dim() > 1:
            nn.init.xavier_uniform_(param)

    return transformer_encoder


# for full transformer
def build_transformer(source_vocab_size: int, target_vocab_size: int, source_seq_len: int, target_seq_len: int, dropouts: Dict[str, float | None] | None, d_model: int=512, d_ff: int=2048,  N: int=6, h: int=8):
    key_list = ['encoder_pe', 'encoder_self_attetion', 'encoder_feed_forward', 'encoder_block', 'decoder_pe', 'decoder_self_attention', 'decoder_cross_attention', 'decoder_feed_forward', 'decoder_block']
    if dropouts is None:
        dropouts = dict()
    for key in key_list:
        if key not in dropouts.keys():
            dropouts[key] = None

    # encoder
    source_embed = InputEmbeddings(d_model, source_vocab_size)
    source_pe = PositionEncoding(d_model, source_seq_len, dropouts['encoder_pe'])
    encoder_layers = []
    for _ in range(N):
        self_attention = MultiHeadAttention(d_model, h, dropouts['encoder_self_attetion'])
        feed_forward = FeedForward(d_model, d_ff, dropouts['encoder_feed_forward'])
        encoder_layers.append(EncoderBlock(self_attention, feed_forward, dropouts['encoder_block']))
    encoder = Encoder(nn.ModuleList(encoder_layers))

    # decoder
    target_embed = InputEmbeddings(d_model, target_vocab_size)
    target_pe = PositionEncoding(d_model, target_seq_len, dropouts['decoder_pe'])
    decoder_layers = []
    for _ in range(N):
        self_attention = MultiHeadAttention(d_model, h, dropouts['decoder_self_attention'])
        cross_attention = MultiHeadAttention(d_model, h, dropouts['decoder_cross_attention'])
        feed_forward = FeedForward(d_model, d_ff, dropouts['decoder_feed_forward'])
        decoder_layers.append(DecoderBlock(self_attention, cross_attention, feed_forward, dropouts['decoder_block']))
    decoder = Decoder(nn.ModuleList(decoder_layers))

    proj = ProjectionLayer(d_model, target_vocab_size)

    transformer = Transformer(encoder, decoder, source_embed, target_embed, source_pe, target_pe, proj)

    # Xavier uniform distribution for parameter initialization
    for param in transformer.parameters():
        if param.dim() > 1:
            nn.init.xavier_uniform_(param)

    return transformer


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, get_scheduler
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

class CodeDataSet(Dataset):
    def __init__(self, data, tokenizer, seq_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        example = self.data[index]
        input_text = "Generate Python: " + example["func_documentation_string"]
        target_text = example["func_code_string"]

        input_encoding = self.tokenizer(input_text, padding="max_length", truncation=True, max_length=self.seq_len, return_tensors="pt")
        target_encoding = self.tokenizer(target_text, padding="max_length", truncation=True, max_length=self.seq_len, return_tensors="pt")

        return {
            "input_ids": input_encoding["input_ids"].squeeze(0),
            "attention_mask": input_encoding["attention_mask"].squeeze(0),
            "labels": target_encoding["input_ids"].squeeze(0),
        }

def transformer_train():
    # dataset: CodeSearchNet
    dataset = load_dataset("code_search_net", "python", trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

    train = CodeDataSet(dataset["train"], tokenizer)
    valid = CodeDataSet(dataset["validation"], tokenizer)

    train_loader = DataLoader(train, batch_size=8, shuffle=True)
    valid_loader = DataLoader(valid, batch_size=8)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    source_vocab_size = tokenizer.vocab_size
    target_vocab_size = tokenizer.vocab_size
    source_seq_len = 512
    target_seq_len = 512
    key_list = ['encoder_pe', 'encoder_self_attetion', 'encoder_feed_forward', 'encoder_block', 'decoder_pe', 'decoder_self_attention', 'decoder_cross_attention', 'decoder_feed_forward', 'decoder_block']
    dropouts = {key: 0.1 for key in key_list}
    model = build_transformer(source_vocab_size, target_vocab_size, source_seq_len, target_seq_len, dropouts).to(device)

    optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_type_id)

    epochs = 3
    num_training_steps = len(train_loader)*epochs
    lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)


    best_loss = float("inf")
    for epoch in range(epochs):
        # train
        model.train()
        train_loss = 0
        for i, batch in enumerate(train_loader):
            input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)
            optimizer.zero_grad()

            # masks
            source_mask = attention_mask.unsqueeze(1).unsqueeze(2).to(device)  # (batch_size, 1, 1, seq_len)
            source_mask = source_mask.repeat(1,8,1,1)
            source_mask = source_mask.to(device)
            target_mask = (labels!=0).unsqueeze(1).unsqueeze(2).to(device)  # (batch_size, 1, 1, seq_len)
            target_len = labels.shape[1]
            target_no_lookahead_mask = torch.tril(torch.ones((target_len, target_len), device=device)).unsqueeze(0).unsqueeze(0)
            target_mask = target_mask.to(torch.bool) & target_no_lookahead_mask.to(torch.bool)
            target_mask = target_mask.to(device)

            out = model(input_ids, labels, source_mask, target_mask)
            loss = loss_fn(out.view(-1, out.shape[-1]), labels.view(-1))
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            train_loss+=loss.item()
            if i%5000==0:
              print(f"Train Completion: {i}/{len(train_loader)}")

        # evaluation
        model.eval()
        valid_loss = 0
        with torch.no_grad():
            for i, batch in enumerate(valid_loader):
                input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)

                # masks
                source_mask = attention_mask.unsqueeze(1).unsqueeze(2).to(device)  # (batch_size, 1, 1, seq_len)
                source_mask = source_mask.repeat(1,8,1,1)
                source_mask = source_mask.to(device)
                target_mask = (labels!=0).unsqueeze(1).unsqueeze(2).to(device)  # (batch_size, 1, 1, seq_len)
                target_len = labels.shape[1]
                target_no_lookahead_mask = torch.tril(torch.ones((target_len, target_len), device=device)).unsqueeze(0).unsqueeze(0)
                target_mask = target_mask.to(torch.bool) & target_no_lookahead_mask.to(torch.bool)
                target_mask = target_mask.to(device)

                out = model(input_ids, labels, source_mask, target_mask)
                loss = loss_fn(out.view(-1, out.shape[-1]), labels.view(-1))
                valid_loss+=loss.item()
                if i%2000==0:
                  print(f"Evaluation Completion: {i}/{len(valid_loader)}")

        print(f"Epoch {epoch+1}: Train Loss = {round(train_loss/len(train_loader), 4)}, Validation Loss = {round(valid_loss/len(valid_loader), 4)}")
        if best_loss > valid_loss:
            best_loss = valid_loss
            # current_dir = os.path.dirname(os.path.abspath(__file__))
            # model_path = os.path.join(current_dir, "codegen_model.pth")
            torch.save(model.state_dict(), "codegen_model.pth")
            print("Best model updated and saved.")

        print()
    print("Training Complete.")

# script
# encoder_transformer_train()
transformer_train()

Train Completion: 0/51523
Train Completion: 5000/51523
Train Completion: 10000/51523
Train Completion: 15000/51523
Train Completion: 20000/51523
Train Completion: 25000/51523
Train Completion: 30000/51523
Train Completion: 35000/51523
Train Completion: 40000/51523
Train Completion: 45000/51523
Train Completion: 50000/51523
Evaluation Completion: 0/2889
Evaluation Completion: 2000/2889
Epoch 1: Train Loss = 5.7489, Validation Loss = 6.0217
Best model updated and saved.

Train Completion: 0/51523
Train Completion: 5000/51523
Train Completion: 10000/51523
Train Completion: 15000/51523
Train Completion: 20000/51523
Train Completion: 25000/51523
Train Completion: 30000/51523
Train Completion: 35000/51523
Train Completion: 40000/51523
Train Completion: 45000/51523
Train Completion: 50000/51523
Evaluation Completion: 0/2889
Evaluation Completion: 2000/2889
Epoch 2: Train Loss = 5.6165, Validation Loss = 6.0183
Best model updated and saved.

Train Completion: 0/51523
Train Completion: 5000/515